# London bike rides: Data exploration and manipulation

In [3]:
# Installing libraries that will be used throughout the analysis
#!pip install pandas
#!pip install zipfile36
#!pip install kaggle

In [4]:
# Importing the Pandas library
import pandas as pd

#Importing the Zipfile library
import zipfile36 as zipfile
    
#Importing the Kaggle library
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [5]:
# Downloading the bike sharing dataset from Kaggle with the API  
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# Extracting the downloaded zip file
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [7]:
# Transforming the CSV file into a Pandas DataFrame
bikes = pd.read_csv('london_merged.csv')

In [8]:
# Overview of the data
bikes.info()

bikes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [9]:
# Counting unique values for the weather_code column
bikes.weather_code.value_counts() 

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [10]:
# Counting unique values for the season column
bikes.season.value_counts() 

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [11]:
# Creating a dictionary to change column names
colnames_dict = {
    'timestamp':'time',
    'cnt':'count',
    't1':'temp_c',
    't2':'apparent_temp_c',
    'hum':'humidity_perc',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

# Renaming the original columns to customized names
bikes.rename(colnames_dict, axis=1, inplace=True)

In [12]:
# Dividing humidity values by 100 in order to convert them to percentages
bikes.humidity_perc = bikes.humidity_perc/100

In [13]:
# Specifying a dictionary to associate the season codes with their actual name
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'autumn',
    '3.0':'winter'
}

# Generating a weather dictionary with the aim of mapping the numbers to their written values
weather_dict = {
    '1.0':'predominantly clear',
    '2.0':'scattered clouds',
    '3.0':'broken clouds',
    '4.0':'cloudy',
    '7.0':'light rain',
    '10.0':'rain with thunderstorm',
    '26.0':'snowfall'
}

In [14]:
# Changing the seasons column data type to string so that season_dict can be used
bikes.season = bikes.season.astype('str')

# Mapping the season codes to their actual name
bikes.season = bikes.season.map(season_dict)

# Changing the weather column data type to then implement weather_dict
bikes.weather = bikes.weather.astype('str')

# Mapping weather nummerical indicators to their literal weather codition
bikes.weather = bikes.weather.map(weather_dict)

In [15]:
# Checking out the dataframe with the intention of corroborating the recent modifications  
bikes.head()

,time,count,temp_c,apparent_temp_c,humidity_perc,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,predominantly clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,predominantly clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,predominantly clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,predominantly clear,0.0,1.0,winter


In [16]:
# Saving the new dataframe into an Excel file that will provide the data for the Tableau viz
bikes.to_excel('london_bikes_final.xlsx', sheet_name = 'data')